In [1]:
import pandas as pd
import sys
import matplotlib.pylab as plt
import operator
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

In [3]:
uid_songid_file_location = './Dataset/byUser.csv'
songid_uid_file_location = './Dataset/bySong.csv'
songid_mappings_file_location = './Dataset/kaggle_songs.txt'
uid_songid_triplets_file_location = './Dataset/kaggle_visible_evaluation_triplets.txt'

In [4]:
uid_songid_file = open(uid_songid_file_location)
uid_songid_file_data = []
for line in uid_songid_file:
    uid_songid_file_data.append(line.strip().split('\t')[1].strip().split(','))
uid_songid_file_data = uid_songid_file_data[1:]
uid_songid_file.close()

In [5]:
songid_mappings_file = open(songid_mappings_file_location)
songid_mappings = {}
for line in songid_mappings_file:
    #print(line)
    temp_var = line.strip().split()
    songid_mappings[temp_var[0]] = int(temp_var[1])

songid_mappings_file.close()
#songid_mappings

In [6]:
#Returns the global frequency of every song.
#A dictionary whose key is the song_id(in numerical format) and its value is the total 
#number of times it was heard by all the users.

def get_global_song_frequency(songid_mappings, song_count_filename):
    
    frequency_dictionary ={}
    song_count_file = open(song_count_filename)
    for data in song_count_file:
        temp_var = data.strip().split('\t')[1:]
        mapped_index = songid_mappings[temp_var[0]]
        if mapped_index in frequency_dictionary:
            frequency_dictionary[mapped_index] += int(temp_var[1])
        else:
            frequency_dictionary[mapped_index] = int(temp_var[1])
            
    return frequency_dictionary

fd = get_global_song_frequency(songid_mappings,uid_songid_triplets_file_location)

In [110]:
def get_songs_with_freq_10_to_5000(all_song_frequency_distribution_dict):
    
    filtered_song_list = []
    for song_id,frequency in all_song_frequency_distribution_dict.items():
        if(frequency>10 and frequency<=5000):
            filtered_song_list.append(song_id)
    
    filtered_song_list.sort()
    return filtered_song_list

filtered_song_list = get_songs_with_freq_10_to_5000(fd)

In [111]:
#In transactional type data, convert hashed song_id to its corresponding numerical song_id 

for itemset in uid_songid_file_data:
    for item_num in range(len(itemset)):
        itemset[item_num] = songid_mappings[itemset[item_num]]   
    itemset.sort()

In [112]:
# Remove those songs from the transaction that are not frequent.

for i in range(len(uid_songid_file_data)):
    itemset = uid_songid_file_data[i]
    for item_num in itemset:
        if item_num not in filtered_song_list:
            itemset.remove(item_num)
            

In [104]:
#Write the reduced dataset(frequency of songs in [11,5000]) in a file 

with open('./Dataset/filtered_uid_songid_dataset_freq_11_to_5000.txt', 'w') as filehandle:
    for itemset in uid_songid_file_data:
        filehandle.write('%s\n' % itemset)